In [12]:
import mediapipe as mp
import cv2
import numpy as np
import math


In [13]:
def analyze_russian_twist(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    twist_count = 0
    twist_direction = None  # "left" or "right" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    shoulder_rotations = []  # Track degree of rotation
    hip_stability = []  # Check if hips remain stable
    back_angles = []  # Check if back maintains proper angle
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate shoulder line (horizontal in rest position)
            shoulder_angle = math.degrees(math.atan2(
                right_shoulder[1] - left_shoulder[1],
                right_shoulder[0] - left_shoulder[0]
            ))
            
            # Store rotation angle
            shoulder_rotations.append(shoulder_angle)
            
            # Calculate back angle (should maintain ~45 degrees)
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, (left_hip[1] + right_hip[1])/2]
            
            # Assuming vertical is 0 degrees, calculate angle of back
            back_angle = math.degrees(math.atan2(
                mid_shoulder[1] - mid_hip[1],
                mid_shoulder[0] - mid_hip[0]
            ))
            back_angles.append(abs(back_angle))
            
            # Check hip stability (hips shouldn't move much)
            hip_pos = [mid_hip[0], mid_hip[1]]
            hip_stability.append(hip_pos)
            
            # Visualize angles
            cv2.putText(annotated_image, 
                        f"Rotation: {shoulder_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Detect twist direction based on shoulder rotation
            # Threshold values would need tuning based on testing
            if shoulder_angle > 15 and (twist_direction == "left" or twist_direction is None):
                twist_direction = "right"
                # Draw twist direction
                cv2.putText(annotated_image, 'RIGHT', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif shoulder_angle < -15 and (twist_direction == "right" or twist_direction is None):
                twist_direction = "left"
                if twist_direction == "right":  # Complete cycle
                    twist_count += 1
                cv2.putText(annotated_image, 'LEFT', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display twist count
        cv2.putText(annotated_image, f'Twists: {twist_count}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "twist_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    max_rotation = max([abs(angle) for angle in shoulder_rotations])
    avg_back_angle = np.mean(back_angles)
    hip_stability_score = 100 - (np.std([pos[0] for pos in hip_stability]) + 
                                np.std([pos[1] for pos in hip_stability])) * 100
    
    # Generate feedback
    feedback = {
        "twist_count": twist_count,
        "form_analysis": {
            "rotation_range": max_rotation,
            "back_angle": avg_back_angle,
            "hip_stability": hip_stability_score,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if max_rotation < 30:
        feedback["feedback"].append("Try to rotate further to each side for a full range of motion.")
    
    if avg_back_angle < 30 or avg_back_angle > 60:
        feedback["feedback"].append("Maintain a back angle of about 45 degrees throughout the exercise.")
    
    if hip_stability_score < 80:
        feedback["feedback"].append("Keep your hips more stable. Twist from your core, not your hips.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your Russian twists show good rotation and stable positioning.")
    
    return feedback

In [14]:
def analyze_situp(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    situp_count = 0
    situp_stage = None  # "up" or "down" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    torso_angles = []  # Track the angle of the torso relative to ground
    neck_angles = []   # Track neck position to ensure proper form
    hip_knee_angles = []  # Check if legs maintain proper position
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate torso angle (shoulder-hip to horizontal)
            # Use a reference horizontal point at the same y as the hip
            hip_horizontal = [left_hip[0] + 0.1, left_hip[1]]  # Point to the right of hip
            torso_angle = calculate_angle(left_shoulder, left_hip, hip_horizontal)
            
            # Adjust angle based on which side of hip the shoulder is
            if left_shoulder[0] < left_hip[0]:  # Shoulder is to the left of hip
                torso_angle = 180 - torso_angle
            
            torso_angles.append(torso_angle)
            
            # Calculate neck angle (check if neck is neutral)
            neck_angle = calculate_angle(left_shoulder, left_ear, [left_ear[0] + 0.1, left_ear[1]])
            neck_angles.append(neck_angle)
            
            # Calculate hip-knee angle
            hip_knee_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            hip_knee_angles.append(hip_knee_angle)
            
            # Visualize angles
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Detect situp stage based on torso angle
            if torso_angle > 70 and (situp_stage == "up" or situp_stage is None):
                situp_stage = "down"
                # Draw situp stage
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif torso_angle < 30 and situp_stage == "down":
                situp_stage = "up"
                situp_count += 1
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Situp #{situp_count} detected at frame with torso angle {torso_angle:.1f}°")
                
        # Display situp count
        cv2.putText(annotated_image, f'Situps: {situp_count}', (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "situp_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    min_torso_angle = min(torso_angles)
    avg_neck_angle = np.mean(neck_angles)
    avg_hip_knee_angle = np.mean(hip_knee_angles)
    
    # Generate feedback
    feedback = {
        "situp_count": situp_count,
        "form_analysis": {
            "min_torso_angle": min_torso_angle,
            "neck_position": avg_neck_angle,
            "leg_position": avg_hip_knee_angle,
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if min_torso_angle > 25:
        feedback["feedback"].append("Try to come up higher in each situp for a full range of motion.")
    
    if avg_neck_angle < 160:
        feedback["feedback"].append("Keep your neck neutral throughout the movement. Avoid pulling with your neck.")
    
    if avg_hip_knee_angle < 70 or avg_hip_knee_angle > 110:
        feedback["feedback"].append("Maintain a consistent knee bend (about 90 degrees) throughout the exercise.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your situps show good full-range motion with proper neck position.")
    
    return feedback

In [15]:
def analyze_side_bend(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    bend_count = 0
    bend_direction = None  # "left" or "right" or "center"
    good_frames = 0
    
    # Lists to store metrics for analysis
    lateral_bends = []  # Track degree of side bend
    forward_leans = []  # Ensure bending is lateral, not forward
    hip_positions = []  # Check if hips remain stable
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, (left_hip[1] + right_hip[1])/2]
            
            # Calculate lateral bend angle
            # Vertical line from mid hip going up
            vertical_ref = [mid_hip[0], mid_hip[1] - 0.1]
            
            # Calculate angle between hip-shoulder line and vertical
            dx = mid_shoulder[0] - mid_hip[0]
            dy = mid_shoulder[1] - mid_hip[1]
            lateral_angle = math.degrees(math.atan2(dx, -dy))  # Negative dy because y increases downward
            lateral_bends.append(lateral_angle)
            
            # Calculate forward/backward lean
            # Need a side view reference for this, using shoulder depth
            shoulder_depth = right_shoulder[0] - left_shoulder[0]  # This works if person faces camera
            forward_leans.append(shoulder_depth)
            
            # Track hip stability
            hip_positions.append(mid_hip)
            
            # Visualize the bend angle
            cv2.putText(annotated_image, 
                        f"Bend: {lateral_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine bend state based on lateral angle
            if lateral_angle > 15 and (bend_direction != "right"):
                bend_direction = "right"
                cv2.putText(annotated_image, 'RIGHT', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif lateral_angle < -15 and (bend_direction != "left"):
                bend_direction = "left"
                cv2.putText(annotated_image, 'LEFT', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif abs(lateral_angle) < 5 and (bend_direction == "right" or bend_direction == "left"):
                if bend_direction == "right" or bend_direction == "left":
                    # Only count when returning to center from a defined side
                    bend_count += 0.5  # Half a rep for each direction
                    bend_direction = "center"
                cv2.putText(annotated_image, 'CENTER', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
        # Display bend count
        cv2.putText(annotated_image, f'Bends: {int(bend_count)}', (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "bend_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    max_left_bend = min(lateral_bends)  # Most negative angle
    max_right_bend = max(lateral_bends)  # Most positive angle
    bend_symmetry = abs(abs(max_left_bend) - max_right_bend) / max(abs(max_left_bend), max_right_bend) * 100
    forward_lean_variation = np.std(forward_leans) * 100  # Higher means more forward/backward motion
    hip_stability = 100 - np.std([p[0] for p in hip_positions] + [p[1] for p in hip_positions]) * 100
    
    # Generate feedback
    feedback = {
        "bend_count": int(bend_count),
        "form_analysis": {
            "max_left_bend": abs(max_left_bend),
            "max_right_bend": max_right_bend,
            "bend_symmetry": 100 - bend_symmetry,  # Higher is better
            "hip_stability": hip_stability,
            "forward_motion": forward_lean_variation,  # Lower is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if max(abs(max_left_bend), max_right_bend) < 15:
        feedback["feedback"].append("Try to bend further to each side for a better stretch.")
    
    if bend_symmetry > 20:
        feedback["feedback"].append("Work on balanced bends to both sides. You're bending more to one side.")
    
    if forward_lean_variation > 10:
        feedback["feedback"].append("Focus on pure side bends. Avoid leaning forward or backward during the movement.")
    
    if hip_stability < 80:
        feedback["feedback"].append("Keep your hips more stable. The movement should come from your waist, not shifting your hips.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your side bends show good range and balance on both sides.")
    
    return feedback

In [16]:
def analyze_bench_press(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    rep_count = 0
    press_stage = None  # "up" or "down" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    elbow_angles = []  # Track elbow angle during movement
    wrist_positions = []  # Track bar path (using wrists as proxy)
    shoulder_stability = []  # Check if shoulders remain on bench
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # For bench press from side view, we'd use one arm's angle
            # From front view, we can use the average of both arms
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elbow_angles.append(avg_elbow_angle)
            
            # Track wrist positions to analyze bar path
            mid_wrist = [(left_wrist[0] + right_wrist[0])/2, (left_wrist[1] + right_wrist[1])/2]
            wrist_positions.append(mid_wrist)
            
            # Track shoulder stability
            shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
            shoulder_stability.append(shoulder_y)
            
            # Visualize the elbow angle
            cv2.putText(annotated_image, 
                        f"Elbow: {avg_elbow_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on elbow angle
            if avg_elbow_angle > 160 and (press_stage == "down" or press_stage is None):
                press_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif avg_elbow_angle < 90 and press_stage == "up":
                press_stage = "down"
                rep_count += 1
                cv2.putText(annotated_image, 'DOWN', (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Bench press rep #{rep_count} detected at frame with elbow angle {avg_elbow_angle:.1f}°")
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
# Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    min_elbow_angle = min(elbow_angles)
    max_elbow_angle = max(elbow_angles)
    
    # Analyze bar path (using wrist positions as proxy)
    wrist_x_positions = [pos[0] for pos in wrist_positions]
    wrist_y_positions = [pos[1] for pos in wrist_positions]
    
    # Calculate horizontal deviation in bar path
    bar_path_deviation = np.std(wrist_x_positions) * 100  # Scaled for readability
    
    # Check shoulder stability
    shoulder_movement = np.std(shoulder_stability) * 100
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "min_elbow_angle": min_elbow_angle,
            "max_elbow_angle": max_elbow_angle,
            "bar_path_stability": 100 - bar_path_deviation,  # Higher is better
            "shoulder_stability": 100 - shoulder_movement,   # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if min_elbow_angle > 100:
        feedback["feedback"].append("Try to lower the bar more for full range of motion. Aim for at least 90° elbow angle at the bottom.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the top of the movement.")
    
    if bar_path_deviation > 5:
        feedback["feedback"].append("Work on a more consistent bar path. Try to keep the bar moving in a straight vertical line.")
    
    if shoulder_movement > 10:
        feedback["feedback"].append("Keep your shoulders stable and in contact with the bench throughout the movement.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your bench press shows good range of motion and stability.")
    
    return feedback

In [17]:
def analyze_lat_pulldown(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    rep_count = 0
    pulldown_stage = None  # "up" or "down" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    elbow_angles = []  # Track elbow flexion
    shoulder_positions = []  # Track if shoulders stay down
    torso_angles = []  # Check for excessive leaning back
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate average elbow angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elbow_angles.append(avg_elbow_angle)
            
            # Track shoulder positions (y coordinate - lower is higher up on screen)
            avg_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
            shoulder_positions.append(avg_shoulder_y)
            
            # Calculate torso angle (mid shoulder to mid hip relative to vertical)
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, (left_hip[1] + right_hip[1])/2]
            
            # Reference point directly below mid_hip (creating a vertical line)
            vertical_ref = [mid_hip[0], mid_hip[1] + 0.1]
            
            torso_angle = calculate_angle(mid_shoulder, mid_hip, vertical_ref)
            if mid_shoulder[0] < mid_hip[0]:  # Leaning left
                torso_angle = 360 - torso_angle
            torso_angles.append(torso_angle)
            
            # Visualize the angles
            cv2.putText(annotated_image, 
                        f"Elbow: {avg_elbow_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, 
                        f"Torso: {torso_angle:.1f}°",
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine pulldown stage based on elbow angle
            if avg_elbow_angle > 150 and (pulldown_stage == "down" or pulldown_stage is None):
                pulldown_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif avg_elbow_angle < 80 and pulldown_stage == "up":
                pulldown_stage = "down"
                rep_count += 1
                cv2.putText(annotated_image, 'DOWN', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Lat pulldown rep #{rep_count} detected at frame with elbow angle {avg_elbow_angle:.1f}°")
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    min_elbow_angle = min(elbow_angles)
    max_elbow_angle = max(elbow_angles)
    
    # Analyze shoulder movement - we want minimal vertical movement
    shoulder_movement = np.max(shoulder_positions) - np.min(shoulder_positions)
    
    # Analyze torso angle - check for excessive leaning back
    max_torso_angle = max(torso_angles)
    avg_torso_angle = np.mean(torso_angles)
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "min_elbow_angle": min_elbow_angle,  # Lower is better (deeper pulldown)
            "max_elbow_angle": max_elbow_angle,  # Higher is better (full extension)
            "shoulder_stability": 100 - (shoulder_movement * 100),  # Higher is better
            "torso_angle": avg_torso_angle,  # Closer to vertical (0) is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if min_elbow_angle > 70:
        feedback["feedback"].append("Try to pull the bar down more completely. Aim to bring it close to your chest.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Allow your arms to fully extend at the top of the movement.")
    
    if shoulder_movement > 0.1:
        feedback["feedback"].append("Keep your shoulders down and back. Avoid shrugging during the exercise.")
    
    if avg_torso_angle > 100 or max_torso_angle > 110:
        feedback["feedback"].append("Maintain a more upright posture. Avoid leaning back excessively during the pulldown.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your lat pulldowns show good range of motion with stable shoulders and torso.")
    
    return feedback

In [18]:
def analyze_cable_row(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    rep_count = 0
    row_stage = None  # "extended" or "retracted" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    elbow_angles = []  # Track elbow angle
    back_angles = []   # Track back angle to ensure upright posture
    shoulder_retraction = []  # Track how far shoulders pull back
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate average elbow angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elbow_angles.append(avg_elbow_angle)
            
            # Calculate back angle relative to vertical
            mid_shoulder = [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2]
            mid_hip = [(left_hip[0] + right_hip[0])/2, (left_hip[1] + right_hip[1])/2]
            
            # Reference point directly below mid_hip (creating a vertical line)
            vertical_ref = [mid_hip[0], mid_hip[1] + 0.1]
            
            back_angle = calculate_angle(mid_shoulder, mid_hip, vertical_ref)
            if mid_shoulder[0] < mid_hip[0]:  # Leaning left
                back_angle = 360 - back_angle
            back_angles.append(back_angle)
            
            # Measure shoulder retraction (x-distance from wrists to shoulders)
            # Lower values mean more retraction (shoulders pulling back relative to wrists)
            left_retraction = abs(left_wrist[0] - left_shoulder[0])
            right_retraction = abs(right_wrist[0] - right_shoulder[0])
            avg_retraction = (left_retraction + right_retraction) / 2
            shoulder_retraction.append(avg_retraction)
            
            # Visualize the angles
            cv2.putText(annotated_image, 
                        f"Elbow: {avg_elbow_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, 
                        f"Back: {back_angle:.1f}°",
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine row stage based on elbow angle and shoulder position
            if avg_elbow_angle > 150 and (row_stage == "retracted" or row_stage is None):
                row_stage = "extended"
                cv2.putText(annotated_image, 'EXTENDED', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif avg_elbow_angle < 80 and row_stage == "extended":
                row_stage = "retracted"
                rep_count += 1
                cv2.putText(annotated_image, 'RETRACTED', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Cable row rep #{rep_count} detected at frame with elbow angle {avg_elbow_angle:.1f}°")
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    min_elbow_angle = min(elbow_angles)
    max_elbow_angle = max(elbow_angles)
    
    # Range of retraction (higher is better)
    retraction_range = max(shoulder_retraction) - min(shoulder_retraction)
    
    # Back angle - should remain consistent and upright
    avg_back_angle = np.mean(back_angles)
    back_angle_variation = np.std(back_angles)
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "min_elbow_angle": min_elbow_angle,  # Lower is better (more bend)
            "max_elbow_angle": max_elbow_angle,  # Higher is better (full extension)
            "retraction_range": retraction_range * 100,  # Higher is better (scaled for readability)
            "back_angle": avg_back_angle,  # Should be close to 0 (upright)
            "back_stability": 100 - (back_angle_variation * 10),  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if min_elbow_angle > 70:
        feedback["feedback"].append("Pull the handle closer to your torso for a more complete row motion.")
    
    if max_elbow_angle < 160:
        feedback["feedback"].append("Extend your arms more fully at the start of each rep.")
    
    if retraction_range < 0.2:
        feedback["feedback"].append("Focus on retracting your shoulder blades at the end of each row for better back engagement.")
    
    if avg_back_angle > 30:
        feedback["feedback"].append("Try to maintain a more upright torso position. You're leaning forward too much.")
    
    if back_angle_variation > 10:
        feedback["feedback"].append("Keep your back angle more consistent throughout the exercise.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your cable rows show good range of motion with proper back positioning.")
    
    return feedback

In [19]:
def analyze_shoulder_press(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    rep_count = 0
    press_stage = None  # "up" or "down" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    arm_extensions = []  # Track how much arms extend
    shoulder_heights = []  # Track shoulder positioning
    elbow_angles = []  # Track elbow angles for symmetry
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elbow_angles.append((left_elbow_angle, right_elbow_angle))
            
            # Track vertical position of wrists relative to shoulders
            # (for measuring arm extension in a press)
            left_extension = left_shoulder[1] - left_wrist[1]  # Positive when wrist is above shoulder
            right_extension = right_shoulder[1] - right_wrist[1]
            avg_extension = (left_extension + right_extension) / 2
            arm_extensions.append(avg_extension)
            
            # Track shoulder height (should stay down, not shrugging)
            avg_shoulder_height = (left_shoulder[1] + right_shoulder[1]) / 2
            shoulder_heights.append(avg_shoulder_height)
            
            # Visualize the angles
            cv2.putText(annotated_image, 
                        f"L Elbow: {left_elbow_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, 
                        f"R Elbow: {right_elbow_angle:.1f}°",
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine press stage based on arm extension
            if avg_extension > 0.15 and (press_stage == "down" or press_stage is None):
                press_stage = "up"
                cv2.putText(annotated_image, 'UP', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif avg_extension < 0.05 and press_stage == "up":
                press_stage = "down"
                rep_count += 1
                cv2.putText(annotated_image, 'DOWN', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Shoulder press rep #{rep_count} detected with arm extension {avg_extension:.3f}")
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    max_extension = max(arm_extensions)
    
    # Calculate arm symmetry (difference between left and right elbow angles)
    angle_diffs = [abs(angles[0] - angles[1]) for angles in elbow_angles]
    avg_asymmetry = np.mean(angle_diffs)
    
    # Shoulder movement (shrugging) - lower variation is better
    shoulder_movement = np.std(shoulder_heights) * 100
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "max_extension": max_extension * 100,  # Higher is better (scaled for readability)
            "arm_symmetry": 100 - avg_asymmetry,  # Higher is better
            "shoulder_stability": 100 - shoulder_movement,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if max_extension < 0.15:
        feedback["feedback"].append("Try to press higher for a full range of motion. Extend your arms more fully overhead.")
    
    if avg_asymmetry > 15:
        feedback["feedback"].append("Work on more balanced pushing. Your arms are pressing unevenly.")
    
    if shoulder_movement > 5:
        feedback["feedback"].append("Keep your shoulders down and stable. Avoid shrugging during the press.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your shoulder press shows good extension with balanced, stable movement.")
    
    return feedback

In [20]:
def analyze_chest_fly(video_path, output_video_path=None):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Setup output video writer if path is provided
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Variables to track exercise state
    rep_count = 0
    fly_stage = None  # "open" or "closed" or None
    good_frames = 0
    
    # Lists to store metrics for analysis
    arm_openings = []  # Track how wide arms open
    elbow_angles = []  # Track elbow angles (should remain consistent)
    shoulder_positions = []  # Check for shoulder stability
    
    print(f"Video dimensions: {frame_width}x{frame_height}, FPS: {fps}")
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        # Convert to RGB for MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Draw pose landmarks on the image
        annotated_image = image.copy()
        
        if results.pose_landmarks:
            good_frames += 1
            
            mp_drawing.draw_landmarks(
                annotated_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            landmarks = results.pose_landmarks.landmark
            
            # Get key points for analysis
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle function
            def calculate_angle(a, b, c):
                a = np.array(a)
                b = np.array(b)
                c = np.array(c)
                
                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                
                if angle > 180.0:
                    angle = 360-angle
                    
                return angle
            
            # Calculate elbow angles (should remain relatively constant in a fly)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
            elbow_angles.append(avg_elbow_angle)
            
            # Measure arm opening angle
            # Use shoulders as reference for the chest plane
            # Calculate the angle between shoulder-wrist lines
            
            # First create vectors from shoulder to wrist
            left_arm_vector = [left_wrist[0] - left_shoulder[0], left_wrist[1] - left_shoulder[1]]
            right_arm_vector = [right_wrist[0] - right_shoulder[0], right_wrist[1] - right_shoulder[1]]
            
            # Calculate the angle between these vectors
            dot_product = left_arm_vector[0] * right_arm_vector[0] + left_arm_vector[1] * right_arm_vector[1]
            left_magnitude = math.sqrt(left_arm_vector[0]**2 + left_arm_vector[1]**2)
            right_magnitude = math.sqrt(right_arm_vector[0]**2 + right_arm_vector[1]**2)
            
            if left_magnitude > 0 and right_magnitude > 0:  # Avoid division by zero
                cos_angle = dot_product / (left_magnitude * right_magnitude)
                # Clamp cos_angle to [-1, 1] to avoid errors due to precision
                cos_angle = max(min(cos_angle, 1.0), -1.0)
                opening_angle = math.degrees(math.acos(cos_angle))
                arm_openings.append(opening_angle)
            
            # Track shoulder stability (y positions)
            shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
            shoulder_positions.append(shoulder_y)
            
            # Visualize the angles
            cv2.putText(annotated_image, 
                        f"Opening: {opening_angle:.1f}°",
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, 
                        f"Elbow: {avg_elbow_angle:.1f}°",
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Determine fly stage based on opening angle
            if opening_angle > 80 and (fly_stage == "closed" or fly_stage is None):
                fly_stage = "open"
                cv2.putText(annotated_image, 'OPEN', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            elif opening_angle < 30 and fly_stage == "open":
                fly_stage = "closed"
                rep_count += 1
                cv2.putText(annotated_image, 'CLOSED', (50, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                print(f"Chest fly rep #{rep_count} detected with opening angle {opening_angle:.1f}°")
                
        # Display rep count
        cv2.putText(annotated_image, f'Reps: {rep_count}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    
        # Write frame to output video
        if output_video_path:
            out.write(annotated_image)
                
    cap.release()
    if output_video_path:
        out.release()
    
    # Only analyze if we have enough valid frames
    if good_frames < 10:
        return {
            "rep_count": 0,
            "error": "Not enough valid pose detections. Check video quality and positioning."
        }
    
    # Calculate metrics for analysis
    max_opening = max(arm_openings)
    min_opening = min(arm_openings)
    opening_range = max_opening - min_opening
    
    # Elbow angle consistency (should stay relatively constant in a fly)
    elbow_variation = np.std(elbow_angles)
    
    # Shoulder stability
    shoulder_movement = np.std(shoulder_positions) * 100
    
    # Generate feedback
    feedback = {
        "rep_count": rep_count,
        "form_analysis": {
            "max_opening": max_opening,  # Higher is better
            "min_opening": min_opening,  # Lower is better
            "range_of_motion": opening_range,  # Higher is better
            "elbow_consistency": 100 - (elbow_variation * 2),  # Higher is better
            "shoulder_stability": 100 - shoulder_movement,  # Higher is better
            "frames_analyzed": good_frames
        },
        "feedback": []
    }
    
    # Add specific feedback based on measurements
    if max_opening < 70:
        feedback["feedback"].append("Try to open your arms wider at the start of the movement for a better chest stretch.")
    
    if min_opening > 40:
        feedback["feedback"].append("Bring your arms closer together at the end of the movement for a full contraction.")
    
    if elbow_variation > 15:
        feedback["feedback"].append("Keep your elbows at a more consistent angle throughout the movement (around 90°).")
    
    if shoulder_movement > 5:
        feedback["feedback"].append("Keep your shoulders pressed against the back pad. Avoid lifting them during the exercise.")
    
    if not feedback["feedback"]:
        feedback["feedback"].append("Great form! Your chest fly shows good range of motion with stable shoulders and consistent elbow position.")
    
    return feedback

In [22]:
# .

# Testing

In [ ]:
# Import common libraries
import mediapipe as mp
import cv2
import numpy as np
import math
import os
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Define exercise analysis functions
# [Your exercise functions go here]

# Testing function
def test_exercise(exercise_function, video_path, output_path=None):
    """
    Test an exercise analysis function and display results
    
    Args:
        exercise_function: The analysis function to test
        video_path: Path to the test video
        output_path: Optional path for the annotated output video
    """
    print(f"Testing {exercise_function.__name__} with video: {video_path}")
    
    # Verify the video exists
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} not found")
        return
    
    # Run the analysis
    result = exercise_function(video_path, output_path)
    
    # Display results
    print("\n=== ANALYSIS RESULTS ===")
    
    if "error" in result:
        print(f"Error: {result['error']}")
        return
    
    # Print rep count
    count_key = next((k for k in result.keys() if k.endswith('_count')), None)
    if count_key:
        print(f"Counted {result[count_key]} reps")
    
    # Print form analysis metrics
    if "form_analysis" in result:
        for key, value in result["form_analysis"].items():
            if isinstance(value, float):
                print(f"{key.replace('_', ' ').title()}: {value:.1f}")
            else:
                print(f"{key.replace('_', ' ').title()}: {value}")
    
    # Print feedback
    if "feedback" in result:
        print("\nFeedback:")
        for item in result["feedback"]:
            print(f"- {item}")
    
    return result

# Test each exercise
# Each of these cells can be run independently to test a specific exercise

# Test Russian Twists
russian_twist_result = test_exercise(
    analyze_russian_twist, 
    "data_da/russian_twist.mov", 
    "data_da/analyzed_russian_twist.mp4"
)

I0000 00:00:1746077225.668793 1305940 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1746077225.768652 1312502 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746077225.781739 1312507 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746077225.800212 1312504 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Testing analyze_russian_twist with video: clips/russian_twist.mov
Video dimensions: 1152x1160, FPS: 60

=== ANALYSIS RESULTS ===
Counted 0 reps
Rotation Range: 173.7
Back Angle: 100.1
Hip Stability: 98.7
Frames Analyzed: 601

Feedback:
- Maintain a back angle of about 45 degrees throughout the exercise.


In [23]:
# Test Sit-ups
situp_result = test_exercise(
    analyze_situp, 
    "data_da/situp.MOV", 
    "data_da/analyzed_situp.mp4"
)

Testing analyze_situp with video: clips/situp.MOV
Video dimensions: 1936x1198, FPS: 60
Situp #1 detected at frame with torso angle 27.7°
Situp #2 detected at frame with torso angle 27.1°
Situp #3 detected at frame with torso angle 28.2°

=== ANALYSIS RESULTS ===
Counted 3 reps
Min Torso Angle: 0.3
Neck Position: 108.8
Leg Position: 78.6
Frames Analyzed: 1027

Feedback:
- Keep your neck neutral throughout the movement. Avoid pulling with your neck.


In [24]:
# Test Side Bends
side_bend_result = test_exercise(
    analyze_side_bend, 
    "data_da/side_bend.MOV", 
    "data_da/analyzed_side_bend.mp4"
)

Testing analyze_side_bend with video: clips/side_bend.MOV
Video dimensions: 1226x1176, FPS: 60

=== ANALYSIS RESULTS ===
Counted 1 reps
Max Left Bend: 33.0
Max Right Bend: 27.7
Bend Symmetry: 83.9
Hip Stability: 85.5
Forward Motion: 3.7
Frames Analyzed: 850

Feedback:
- Great form! Your side bends show good range and balance on both sides.


In [25]:
# Test Bench Press
bench_press_result = test_exercise(
    analyze_bench_press, 
    "data_da/bench_press.MOV", 
    "data_da/analyzed_bench_press.mp4"
)

Testing analyze_bench_press with video: clips/bench_press.MOV
Video dimensions: 1734x1142, FPS: 60
Bench press rep #1 detected at frame with elbow angle 61.3°

=== ANALYSIS RESULTS ===
Counted 1 reps
Min Elbow Angle: 16.0
Max Elbow Angle: 163.2
Bar Path Stability: 86.9
Shoulder Stability: 93.3
Frames Analyzed: 404

Feedback:
- Work on a more consistent bar path. Try to keep the bar moving in a straight vertical line.


In [26]:
# Test Lat Pulldowns
lat_pulldown_result = test_exercise(
    analyze_lat_pulldown, 
    "data_da/lat_pulldown.MOV", 
    "data_da/analyzed_lat_pulldown.mp4"
)

Testing analyze_lat_pulldown with video: clips/lat_pulldown.MOV
Video dimensions: 952x1148, FPS: 60
Lat pulldown rep #1 detected at frame with elbow angle 77.0°
Lat pulldown rep #2 detected at frame with elbow angle 79.5°
Lat pulldown rep #3 detected at frame with elbow angle 78.6°

=== ANALYSIS RESULTS ===
Counted 3 reps
Min Elbow Angle: 22.3
Max Elbow Angle: 177.8
Shoulder Stability: 80.0
Torso Angle: 153.5
Frames Analyzed: 551

Feedback:
- Keep your shoulders down and back. Avoid shrugging during the exercise.
- Maintain a more upright posture. Avoid leaning back excessively during the pulldown.


In [27]:
# Test Cable Rows
cable_row_result = test_exercise(
    analyze_cable_row, 
    "data_da/cable_row.MOV", 
    "data_da/analyzed_cable_row.mp4"
)

Testing analyze_cable_row with video: clips/cable_row.MOV
Video dimensions: 1606x1148, FPS: 60
Cable row rep #1 detected at frame with elbow angle 77.9°
Cable row rep #2 detected at frame with elbow angle 71.4°

=== ANALYSIS RESULTS ===
Counted 2 reps
Min Elbow Angle: 68.5
Max Elbow Angle: 160.1
Retraction Range: 18.8
Back Angle: 187.2
Back Stability: -33.2
Frames Analyzed: 468

Feedback:
- Focus on retracting your shoulder blades at the end of each row for better back engagement.
- Try to maintain a more upright torso position. You're leaning forward too much.
- Keep your back angle more consistent throughout the exercise.


In [28]:
# Test Shoulder Press
shoulder_press_result = test_exercise(
    analyze_shoulder_press, 
    "data_da/shoulder_press.MOV", 
    "data_da/analyzed_shoulder_press.mp4"
)

Testing analyze_shoulder_press with video: clips/shoulder_press.MOV
Video dimensions: 1120x1148, FPS: 60
Shoulder press rep #1 detected with arm extension 0.047
Shoulder press rep #2 detected with arm extension 0.050
Shoulder press rep #3 detected with arm extension 0.048
Shoulder press rep #4 detected with arm extension 0.042

=== ANALYSIS RESULTS ===
Counted 4 reps
Max Extension: 30.6
Arm Symmetry: 71.5
Shoulder Stability: 98.9
Frames Analyzed: 743

Feedback:
- Work on more balanced pushing. Your arms are pressing unevenly.


In [29]:
# Test Chest Fly
chest_fly_result = test_exercise(
    analyze_chest_fly, 
    "data_da/chest_fly.MOV", 
    "data_da/analyzed_chest_fly.mp4"
)

Testing analyze_chest_fly with video: clips/chest_fly.MOV
Video dimensions: 2284x1076, FPS: 60
Chest fly rep #1 detected with opening angle 28.0°
Chest fly rep #2 detected with opening angle 22.3°
Chest fly rep #3 detected with opening angle 29.9°
Chest fly rep #4 detected with opening angle 8.1°
Chest fly rep #5 detected with opening angle 27.5°

=== ANALYSIS RESULTS ===
Counted 5 reps
Max Opening: 163.9
Min Opening: 0.0
Range Of Motion: 163.9
Elbow Consistency: 30.6
Shoulder Stability: 99.1
Frames Analyzed: 1409

Feedback:
- Keep your elbows at a more consistent angle throughout the movement (around 90°).
